# RAG tool that adds transparency to political bias in top stories, by site

Step 1: Installing libraries and adding keys

In [135]:
import warnings
warnings.filterwarnings('ignore')

In [136]:
!pip install -q pillow==10.0.1
!pip install -q unstructured-client unstructured[all-docs]==0.12.5 langchain transformers accelerate bitsandbytes sentence-transformers faiss-gpu

In [137]:
!apt-get -qq install poppler-utils tesseract-ocr

In [148]:
!pip install -q langchain_community

In [149]:
!pip install -q openai

In [163]:
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.6 MB/s eta 0:00:00


In [150]:
import os
from google.colab import userdata

In [139]:
from getpass import getpass
unstructered_api_key = getpass('Enter Unstructured API Key')

Enter Unstructured API Key··········


In [142]:
# taking this from above for unstructured
os.environ["UNSTRUCTURED_API_KEY"] = unstructered_api_key

In [143]:
# Setup client
from unstructured_client import UnstructuredClient

unstructured_api_key = os.environ.get("UNSTRUCTURED_API_KEY")

client = UnstructuredClient(
    api_key_auth=unstructured_api_key,
)

In [144]:
# Enter rest of keys
serper_key = getpass('Enter Serper API Key')
hugging_face_api_key = getpass('Enter HuggingFace API Key')

Enter Serper API Key··········
Enter HuggingFace API Key··········


In [145]:
# assign keys
os.environ["SERPER_KEY"] = serper_key
os.environ["HF_API_KEY"] = hugging_face_api_key

In [146]:
SERPER_KEY = os.environ["SERPER_KEY"]
HF_API_KEY = os.environ["HF_API_KEY"]

In [147]:
# do the same for OAI API key
oai_key = getpass('Enter OAI key')

Enter OAI key··········


In [174]:
# Import libraries
import os
from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError
from unstructured.staging.base import dict_to_elements
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import OpenAI, VectorDBQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader, JSONLoader, UnstructuredFileLoader, WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import LLMChain, SequentialChain, RetrievalQA
from langchain.memory import VectorStoreRetrieverMemory
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import Document, StrOutputParser
from bs4 import BeautifulSoup
import faiss
import json
from pathlib import Path
from pprint import pprint
import requests
from unstructured.cleaners.core import clean_extra_whitespace
from langchain_community.document_loaders import UnstructuredPDFLoader

In [159]:
# PDF for RAG to define liberal and conservative text in language
path_to_pdf=("/content/Flanker paper.PolPsychVersion.04-27-11.pdf")

with open(path_to_pdf, "rb") as f:
  files=shared.Files(
      content=f.read(),
      file_name=path_to_pdf,
      )
  req = shared.PartitionParameters(
    files=files,
    chunking_strategy="by_title",
    max_characters=512,
  )
  try:
    resp = client.general.partition(req)
  except SDKError as e:
    print(e)

elements = dict_to_elements(resp.elements)

In [164]:
documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    documents.append(Document(page_content=element.text, metadata=metadata))

db = FAISS.from_documents(documents, OpenAIEmbeddings(openai_api_key=oai_key))
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [58]:
# define variables - from UI so placholders
search_query = 'BBC'
#num_results = 5
personal_political_assessment = "5"

In [48]:
# define model
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo', max_tokens=1024, openai_api_key=oai_key)
search = GoogleSerperAPIWrapper(type="news", tbs="qdr:w1", serper_api_key=SERPER_KEY)
result_dict = search.results(search_query)
token_limit=8000

In [166]:
for i, item in zip(range(num_results), result_dict['news']):
    url = item.get('link','N/A')
    loader = WebBaseLoader(url)
    article = loader.load()

In [169]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [129]:
# First Chain - assess the article in a chain for langchain
first_template = ChatPromptTemplate.from_template("""You are a university political science professor who uses brevity and inherent logic to assess any political bias from language in a news article {article}. Summarize the article in as few words as possible, being extremely clear. Follow the steps below to output an analysis in one brief sentence. Then list For Potential Logical Fallacies, return a short numbered list of logical fallacies as defined by Aristotle ranked in order of impact on society and ability to mislead, explain why it is a logical fallacy briefly. Do not list any more than 3. If no fallacies are present, return a brief note that no major logical fallacies are found.
Return output using the labels below, with a new line of text directly under each label.\\
and rate it on a scale of between 1 and 10, with 1 being extremely conservative and 10 being extremely liberal. \
After assessing the article, normalize this in relation to a defined personal political assessment with value {personal_political_assessment} for an audience defines their own political beliefs, on a scale of 1 to 10 with 1 being extremely conservative and 10 being extremely liberal, and explain the strongest reason for the score and its scale in a brief sentence \
So for example if the user is defined as a 5 then no difference should be inherent in the assessment of the political bias from the news article.  \
If the user is defined as a 3 then the political bias assessed in the news article should be increased by a factor of 2. \
If a user is defined as a 7 then the political bias assessed in the news article should be decreased by a factor of 2. \
Do not explain anything in the first person, just assess in one sentence based on the values and return the result and explain the reason for the political bias assessment.
Summary:
Potential Political Bias Assessment on a scale of 1 to 10:
Reason for Political Bias:
Potential Logical Fallacies:
""")
#prompt_template1 = PromptTemplate(input_variables=["article", "personal_political_assessment"], template=first_template)
first_chain = first_template | llm | StrOutputParser()
#first_chain = LLMChain(llm=llm, prompt=first_chain, output_key='Assessment')

In [130]:
out = first_chain.invoke({"article": article, "personal_political_assessment": personal_political_assessment})
print(out)

Summary: The article discusses how atoll islands like the Maldives have not shrunk despite rising sea levels, offering hope for their future.

Potential Political Bias Assessment on a scale of 1 to 10: 3
Reason for Political Bias: The article focuses on scientific research and environmental issues without displaying any overt political bias.

Potential Logical Fallacies: 
1. False Cause - implying that rising sea levels are not a threat to atoll islands based on limited data.
2. Hasty Generalization - drawing conclusions about all atoll islands based on a study of a limited number of islands.
3. Appeal to Authority - relying heavily on the opinions of scientists without considering alternative perspectives.
